In [1]:
%pip install ninjabook tabulate --force-reinstall

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.2/217.2 kB 6.8 MB/s eta 0:00:00
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: ninjabook
    Found existing installation: ninjabook 0.1.5
    Uninstalling ninjabook-0.1.5:
      Successfully uninstalled ninjabook-0.1.5

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Parsing raw data into `Event`

In [10]:
import csv
from ninjabook import Event

events = []
with open("../data/norm_book_data_300k.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        event = Event(
            timestamp= int(row['timestamp']),
            seq=int(row['seq']),
            is_trade=bool(row['is_trade']=='1'),
            is_buy=bool(row['is_buy']=='1'),
            price=float(row['price']),
            size=float(row['size'])
        )
        events.append(event)

len(events)

300000

# Ingesting `Events` and streaming BBO

In [11]:
from ninjabook import Orderbook 
from tabulate import tabulate 

ob = Orderbook(0.01)

data = []

for event in events[:200_000]:
    bbo = ob.process_stream_bbo(event)
    if bbo != None:
        best_bid_px = bbo[0].price if bbo[0] is not None else None
        best_bid_sz = bbo[0].size if bbo[0] is not None else None
        best_ask_px = bbo[1].price if bbo[1] is not None else None
        best_ask_sz = bbo[1].size if bbo[1] is not None else None
        mid = ob.midprice()
        data.append([ob.last_updated, ob.last_sequence, best_bid_sz, best_bid_px, mid, best_ask_px, best_ask_sz])

headers = ["ts", "seq", "best_bid_sz", "best_bid_px", "mid", "best_ask_px", "best_ask_sz"]

print(tabulate(data[-5:], headers=headers, tablefmt="grid"))


+------------------+--------+---------------+---------------+---------+---------------+---------------+
|               ts |    seq |   best_bid_sz |   best_bid_px |     mid |   best_ask_px |   best_ask_sz |
+==================+========+===============+===============+=========+===============+===============+
| 1575161162310275 | 199955 |      0.019027 |       7490.55 | 7491.39 |       7492.22 |      0.204504 |
+------------------+--------+---------------+---------------+---------+---------------+---------------+
| 1575161162409739 | 199968 |      0.019027 |       7490.55 | 7491.35 |       7492.16 |      0.019353 |
+------------------+--------+---------------+---------------+---------+---------------+---------------+
| 1575161162509896 | 199980 |      0.019027 |       7490.55 | 7491.33 |       7492.1  |      0.3      |
+------------------+--------+---------------+---------------+---------+---------------+---------------+
| 1575161162610363 | 199987 |      0.019027 |       7490.55 | 74

# Processing `Events` and getting top 3 bids and asks

In [12]:
from ninjabook import Orderbook
from tabulate import tabulate 

ob = Orderbook(0.01)

for event in events[:200_000]:
    ob.process(event)
    
data = []

for ask in sorted(ob.top_asks(3), key=lambda x: x.price, reverse=True):
    data.append([None, None, None, ask.price, ask.size])
    
data.append([None, None, ob.midprice(), None, None])

for bid in ob.top_bids(3):
    data.append([bid.size, bid.price, None, None, None])
    
headers = ["best_bid_sz", "best_bid_px", "mid", "best_ask_px", "best_ask_sz"]

print("last sequence:", ob.last_sequence)
print(tabulate(data, headers=headers, tablefmt="grid"))


last sequence: 199999
+---------------+---------------+---------+---------------+---------------+
|   best_bid_sz |   best_bid_px |     mid |   best_ask_px |   best_ask_sz |
+===============+===============+=========+===============+===============+
|               |               |         |       7492.02 |      4        |
+---------------+---------------+---------+---------------+---------------+
|               |               |         |       7492.01 |      0.149845 |
+---------------+---------------+---------+---------------+---------------+
|               |               |         |       7491.99 |      0.214587 |
+---------------+---------------+---------+---------------+---------------+
|               |               | 7491.27 |               |               |
+---------------+---------------+---------+---------------+---------------+
|      0.019027 |       7490.55 |         |               |               |
+---------------+---------------+---------+---------------+-------

# Processing raw events without using `Event`

In [9]:
from tabulate import tabulate 

from ninjabook import Orderbook


ob = Orderbook(0.01) # tick_size = 0.01

for raw in events:
    ob.process_raw(
        raw.timestamp, 
        raw.seq, 
        raw.is_trade, 
        raw.is_buy, 
        raw.price, 
        raw.size
    )
    
data = []

for ask in sorted(ob.top_asks(3), key=lambda x: x.price, reverse=True):
    data.append([None, None, None, ask.price, ask.size])
    
data.append([None, None, ob.midprice(), None, None])

for bid in ob.top_bids(3):
    data.append([bid.size, bid.price, None, None, None])
    
headers = ["best_bid_sz", "best_bid_px", "mid", "best_ask_px", "best_ask_sz"]

print("last sequence:", ob.last_sequence)
print(tabulate(data, headers=headers, tablefmt="grid"))


last sequence: 299999
+---------------+---------------+---------+---------------+---------------+
|   best_bid_sz |   best_bid_px |     mid |   best_ask_px |   best_ask_sz |
+===============+===============+=========+===============+===============+
|               |               |         |       7451.03 |       2.57563 |
+---------------+---------------+---------+---------------+---------------+
|               |               |         |       7451.02 |       1.11018 |
+---------------+---------------+---------+---------------+---------------+
|               |               |         |       7451.01 |       3.9983  |
+---------------+---------------+---------+---------------+---------------+
|               |               | 7450.36 |               |               |
+---------------+---------------+---------+---------------+---------------+
|      0.00279  |       7449.71 |         |               |               |
+---------------+---------------+---------+---------------+-------